# Introduction to Bioinformatics

## 02 - Pairwise alignment

In the first practice session, we learned how to access a file's content and how to do some basic operations with python strings. We worked with a fasta-format file that contained only one sequence. We pick up from this last point and extend our method to read multiple sequences from a fasta file. Later, we explore how a specialized library called [Biopython](https://biopython.org/) deals with sequence information. Using this library, we then compare many sequences utilizing the percentage of identity as a metric.

### Multiple sequence data

Most of the time, we will want to compare multiple sequences to know how they relate to each other. When these sequences are evolutionarily related, they are known as homologous sequences. Now that we are familiar with single sequence fasta files let's try to read a multiple sequence fasta file into a dictionary.

To obtain the data, we will [BLAST](https://en.wikipedia.org/wiki/BLAST_(biotechnology)) our sequence and download the resulting fasta file. We will run the BLAST algorithm using the P0CG48 code inside the Uniprot page:

[Uniprot BLAST page](https://www.uniprot.org/blast/)

How many sequences do we have?

In [ ]:
count = 0
with open('input/P0CG48_BLAST_sequences.fasta') as ff:
    for l in ff:        
        # Gather sequence lines only
        if l.startswith('>'):
#             count = count + 1
            count += 1
print(count)

Now we put the fasta file into a dictionary:

In [ ]:
# Define empty dictionary to store sequences in it
sequences = {}

# Define the single-sequence container as an empty string
sequence = ''
# Open the multiple sequences fasta file
with open('input/P0CG48_BLAST_sequences.fasta') as ff:
    for l in ff:        
        # Gather sequence lines only
        if not l.startswith('>'):
            # Merge these lines into a single string
            sequence += l.strip()
        # Match fasta name lines (starting with >)
        else:
            # Not at the first iteration because sequence must different than ''
            if sequence != '':
                # Save entry in the dictionary
                sequences[uniprot_id] = sequence
            uniprot_id = l.split('|')[1]
            # Empty the sequence string 
            sequence = ''
            
# Print the dictionary length (the number of elements in it).
print(len(sequences.keys()))

We notice that we are short on one sequence. Why is that?

After the final iteration, there is still a missing entry in the dictionary, and this is because the final iteration does not store the last iterated sequence. We only save the sequence we were gathering when we find a ">" symbol at the beginning of the line. However, our last sequence is not followed by this symbol. This pitfall is easily managed if we tell python to store the end sequence once it has done its last iteration, i.e., after the "for loop" has finished.

In [ ]:
sequences = {}

sequence = ''
with open('input/P0CG48_BLAST_sequences.fasta') as ff:
    for l in ff:        
        if not l.startswith('>'):
            sequence += l.strip()
        else:
            if sequence != '':
                sequences[uniprot_id] = sequence
            uniprot_id = l.split('|')[1]
            sequence = ''
            
    # Notice where the identation is located. It will save the last sequence after
    # the "for loop" has finished; this indentation is outside the for loop.
    sequences[uniprot_id] = sequence

# Print the dictionary entries
print(len(sequences.keys()))

In [ ]:
# Let's print our dictionary to see how it looks like.
print(sequences)

### Declaring python functions

Note that the previous code is sufficiently generic to be applied to any fasta file. You can reuse the code as many times as you want if you wrap it as a Python function. If the functions you write are generalizable, they can be applied in many different contexts, saving us a lot of typing!

We will convert the above code into a function. This function will get as input the path to the fasta file, and it will return a dictionary containing the sequences. Also, the function must have proper documentation. The documentation should include the definition and usage of the function. In this way, the function can be understood and used by others.

To declare a function, we begin with the word "def," followed by the function's name, inputs, and keywords. The function ends when it encounters a return statement. This statement allows you to store the function's output into a variable after its execution.

```
def myfunction(arguments, keywords=None):
    """
    Here goes the documentation
    """
    
    Here goes the code
    
    Here goes the return statement
    
```  
     
Let's declare a function to read fasta files as dictionaries:

In [ ]:
def readFasta(input_fasta):
    """
    Reads a fasta file and returns the sequence data as a dictionary.
    
    Parameters
    ----------
    input_fasta : str
        Path to the input fasta file
    
    Returns
    -------
    sequences : dict
        Dictionary containing the IDs (keys) and squences (values) in the fasta file.
    """
    sequences = {}
    sequence = ''
    with open(input_fasta) as ff:
        for l in ff:        
            if not l.startswith('>'):
                sequence += l.strip()
            else:
                if sequence != '':
                    sequences[uniprot_id] = sequence
                uniprot_id = l.split('|')[1]
                sequence = ''
        sequences[uniprot_id] = sequence
    
    return sequences

In [ ]:
sequences = readFasta('input/P0CG48_BLAST_sequences.fasta')
print(sequences.keys())

This syntax is a compelling way of programming. You can reuse any code previously made and put it to fair use in different contexts by making functions.

To get help for a function, you just call:

In [ ]:
help(readFasta)

This prints out the documentation of the function. Acceptable documentation should give you all you need to understand how the function is used. To get specific details of a function, you need to look at its code and comments within.

### Read sequences with Biopython 

Now that you are aware of how files can be read and parsed by python. We will move forward and use a specialized library to read, store and analyze our data. It uses more sophisticated functions to read and parse fasta files and other file formats relevant for storing biological data.

Let's read our blast fasta file with Biopython. We start by importing Biopython and all other libraries we will need.

In [ ]:
from Bio import SeqIO

Biopython has a SeqIO interface that allows us to read fasta formatted files. We use the method parse to read the data into a generator, a particular type of iterator.

In [ ]:
# Read fasta file with Biopython's SeqIO interface
sequences = SeqIO.parse('input/P0CG48_BLAST_sequences.fasta', 'fasta')

#We print the sequences contained in the iterator with a for loop
for seq_record in sequences:
    print(seq_record)
    print() # Empty print to separate each iteration's print

Now we use the help() method to understand how Biopython represents sequences.

In [ ]:
# Let us call help in the sequence object
help(seq_record)

Two main attributes represent the id and the sequences.  Additionally, other attributes can contain additional information about our sequence. Let's print some attributes to see what they have.

In [ ]:
# Sequence information
print(seq_record.id)
print(seq_record.name)
print(seq_record.description)
print(seq_record.annotations)
print('----------')

How do we access the sequence?

In [ ]:
# Sequence itself
print(seq_record.seq)
print()
print(seq_record[10:20]) # Slice directly on the SeqRecord object
print()
print(seq_record.seq[10:20]) # Slice on the Seq object
print('----------')
# Print types
print(type(seq_record))
print(type(seq_record[10:20]))
print('----------')
print(type(seq_record.seq))
print(type(seq_record.seq[10:20])) 
print('----------')

We can call help on the Seq type object to know more about it.

In [ ]:
help(seq_record.seq)

To learn more about how Biopython handles sequence files, please visit:

[Biopython SeqIO](https://biopython.org/wiki/SeqIO)

### Align sequences with biopython

Biopython contains methods to align a pair of sequences, either globally or locally.
Let's pick up two sequences and align them:

In [ ]:
from Bio.Seq import Seq

In [ ]:
# These are two random DNA sequences of 20bp length
seq1 = Seq('GGGCGGCCTGGCCGGAGCAC')
seq2 = Seq('TCCTCGGCGATGTGTGTAAA')

print(type(seq1))
print(type(seq2))

We will employ the Bio.pairwise2 module to align the two sequences globally:

In [ ]:
from Bio import pairwise2

In [ ]:
# align and print the number of different alignments
alignments = pairwise2.align.globalxx(seq1, seq2)
print(len(alignments))

The method generates many alignments that have similar scores. We can access all of them by iterating the alignment object. We do so and print the resulting five first alignments

In [ ]:
# Print first 5 alignments
for a in alignments[:5]:
    print(a)

We can improve the visualization by using the "format_alignment" function of the Biopython pairwise2 method.

In [ ]:
from Bio.pairwise2 import format_alignment

In [ ]:
print('Pretty print first alignment:')
print(format_alignment(*alignments[0]))
print('Print alignment sequences:')
# We access each sequence with indexes
print(alignments[0][0]) # first sequence with index 0
print(alignments[0][1]) # second sequence with index 1

We see that for each alignment generated, we can access the individual strings of each sequence by just using indexing over the alignment object.

Now using this, let's calculate the percentage of identity for each alignment. For this, we create a function:

In [ ]:
def calculatePID(alignment):
    
    # Get the two sequences
    seq1 = alignment[0]
    seq2 = alignment[1]
    
    count = 0 # Variable containing the count of matching characters
    # iterate
    for j in range(len(seq1)):
        p1 = seq1[j] # Position i of first alignment 
        p2 = seq2[j] # Position i of second alignment
    
        # Only count when we don't have '-' at any position.
        if p1 != '-' and p2 != '-':
            # Only when the sequences match
            if p1 == p1:
                count += 1
            
    # Get length of each sequence
    l1 = len([p for p in seq1 if p != '-'])
    l2 = len([p for p in seq2 if p != '-'])
    
    # the PID uses the length of the shortest sequence to convert to percentage
    pid = count/min(l1, l2)
    return pid

Using this function, let's calculate the percentage of identity (PID) of an alignment:

In [ ]:
print('Alignment 0 PID:')
print(calculatePID(alignments[0]))

### Calculate a PID matrix

We now move into comparing sequences in an all vs. all fashion. This type of comparison is advantageous if we want to find groups of related sequences. To this end, We need to compare every sequence to each other. We will need a matrix to store the comparison data (NumPy offers a convenient way of storaging matrix data). Let's have a look at how a matrix of data looks like:

In [ ]:
import numpy as np

In [ ]:
# print a 5x5 matrix filled with zeros
M = np.zeros((5,5))
print(M)

Let's play to fill some entries

In [ ]:
M[1][2] = 1 # i<j, upper right 
M[3][1] = 2 # i>j, lower left
M[3][3] = 3 # i=j, diagonal
print(M)

Each (i,j) element of the matrix will contain a value that corresponds to the comparison of elements i and j.

We now read a set of N sequences and store them in a dictionary:

In [ ]:
# Read the sequences in the file
sequences = SeqIO.parse('input/apolipoproteinCI_sequences.fasta', 'fasta')
# Convert iterator to dictionary
sequences = {s.id : s for s in sequences}

We will calculate the percentage of identity between each pair of sequences
and store the M matrix values. First, we initialize a matrix with the correct shape and filled with zeros to keep all the values (NxN).

In [ ]:
# Initialize the NxN matrix
N = len(sequences)
M = np.zeros((N, N))
print(M.shape)
print(M)

Finally, we compare all the sequences to each other. To iterate the elements to compare, we use a double for loop; one loop for i and one for j elements:

In [ ]:
for i,s1 in enumerate(sequences):
    for j, s2 in enumerate(sequences):
        seq1 = sequences[s1]
        seq2 = sequences[s2]
        if i == j: # the same element always has a PID=1.0
            M[i][j] = 1.0
        if j > i : # We compare only half matrix
            alignment = pairwise2.align.globalxx(sequences[s1], sequences[s2])
            M[i][j] = calculatePID(alignment[0])
            M[j][i] = M[i][j] # This to fill the lower half matrix

Let's visualize how our matrix looks like using matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.matshow(M)
plt.title('PID matrix for the Apolipoprotein CI')
plt.xlabel('Sequence index i')
plt.ylabel('Sequence index j')

We can get the centroid structure by printing the column average 

In [ ]:
print('Average PID value')
print(np.average(M))
print()
print('Average PID value for each sequence:')
print(np.average(M, axis=0))
print()
print('Maximum PID value for each sequence:')
print(np.max(np.average(M, axis=0)))
print()
print('The index of the centroid sequence is:')
centroid_index = np.argmax(np.average(M, axis=0))
print(centroid_index)
print()
print('The centroid sequence is:')
print(list(sequences.keys())[centroid_index])

### Wrapping up

In this second practice session, we learned:

- How to read multiple sequences from a file and store them as a dictionary
- How to write a Python function for repetitive tasks
- How to read sequences with Biopython 
- How to do a pairwise alignment of two sequences with Biopython
- How to calculate a PID matrix